# Funções, bibliotecas e Dataframe ficticios

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.font_manager as fm #para alterar as fontes dos graficos
import seaborn as sns

import pyperclip

# Definindo o estilo de fonte e tema
fonte = fm.FontProperties(family="Calibri", style="italic")

sns.set_style('whitegrid') #cor da grade(fundo)

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
import requests

# CARREGANDO BASE
arquivo = 'fake_database'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
print(url)
#response = requests.get(url); code = response.text; exec(code)
#df = fake_database2(2250); df_bck = df.copy(); display(df.head())

arquivo = 'funcoes_estatisticas'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
print(url)
response = requests.get(url); code = response.text; exec(code)

# Função para avaliação de modelos exibindo metricas de avaliação
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay
arquivo = 'ML_supervised_learning'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
print(url); response = requests.get(url); code = response.text; exec(code)

print('TUDO OK')

https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/fake_database.py
https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/funcoes_estatisticas.py
https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/ML_supervised_learning.py
TUDO OK


In [4]:
def contribuicao_proba_tabela(df, coluna_analisada, model, sep=10):
    # Criar DataFrame para análise com a variável de interesse variando em um intervalo
    minimo, maximo = df[coluna_analisada].min(), df[coluna_analisada].max()
    df_analise_feature = pd.DataFrame()

    # Preencher as demais colunas com o valor médio
    for coluna in df.columns:
        if coluna != coluna_analisada:
            df_analise_feature[coluna] = df[coluna].mean()
        else:
            df_analise_feature[coluna] = np.linspace(minimo, maximo, sep)

    # Fazer previsão de probabilidade e adicionar ao DataFrame de análise
    df_analise_feature['predict_proba'] = model.predict_proba(df_analise_feature)[:, 1]

    # Retornar apenas a coluna analisada e a probabilidade prevista
    return df_analise_feature[[coluna_analisada, 'predict_proba']]

#x_data = pd.DataFrame(x_teste, columns=df.columns[:-1])
#resultado = contribuicao_proba_tabela(x_data, 'feature_4', modelo_m1, sep=10)
#resultado

def contribuicao_proba_grafico(resultado, coluna):
    cmap = plt.get_cmap('viridis')
    norm = plt.Normalize(resultado['predict_proba'].min(), resultado['predict_proba'].max())
    colors = cmap(norm(resultado['predict_proba']))

    ax = sns.lineplot(x=resultado[coluna].round(1), y=resultado['predict_proba'])

#plt.figure(figsize=(4,3))
#contribuicao_proba_grafico(resultado, 'feature_4')
#plt.show()

def contribuicao_proba(x_data, modelo, sep=10):
    for i, coluna_analisada in enumerate(x_data.columns, 1):
        plt.subplot(10, 3, i)
        resultado = contribuicao_proba_tabela(x_data, coluna_analisada, modelo, sep=sep)
        contribuicao_proba_grafico(resultado, coluna_analisada)


from sklearn.inspection import PartialDependenceDisplay
def contribuicao_proba2(x_data, modelo):
    # Definir o tamanho da figura geral
    fig, axes = plt.subplots(len(x_data.columns)//3+1, 3, figsize=(10, 10))
    axes = axes.ravel()  # Transformar a matriz de eixos em uma lista para fácil iteração

    # Iterar pelas colunas e gerar gráficos de dependência parcial
    for i, column in enumerate(x_data.columns):
        PartialDependenceDisplay.from_estimator(modelo, x_data, [column], ax=axes[i])

    # Ajustar o layout para evitar sobreposição
    plt.tight_layout()
    plt.show()

# Criando dataframe sintetico desbalanceado

In [5]:
from sklearn.datasets import make_classification

# Gerar dados sintéticos desbalanceados
x, y = make_classification(n_samples=7346, 
                           n_features=10, # qtd total de features
                           n_informative=6, # features realmente uteis para o modelo
                           n_redundant=3, # features que diz praticamente oque uma outra já fim
                           n_classes=2, 
                           n_clusters_per_class=3,  # Subgrupos
                           weights=[0.95, 0.05], 
                           class_sep=0.125,
                           random_state=3141592)

# Criar um DataFrame
df = pd.DataFrame(x, columns=[f'feature_{i}' for i in range(x.shape[1])])
df['target'] = y

# Ver a distribuição de classes
print(df['target'].value_counts(normalize=True))
print(df['target'].value_counts())

display(pd.concat([df.head(5), df.tail(5)]).T)
# Salvar para análise
#df.to_csv('imbalanced_dataset.csv', index=False)

target
0    0.945685
1    0.054315
Name: proportion, dtype: float64
target
0    6947
1     399
Name: count, dtype: int64


,0,1,2,3,4,7341,7342,7343,7344,7345
feature_0,-1.043680,0.024369,0.615455,-0.085438,-1.950661,-0.517000,-2.692529,-1.076198,-0.173833,0.051514
feature_1,-1.990359,1.052022,-0.135408,-0.087211,-0.684208,0.062871,-2.081917,-0.133125,-0.621593,1.548708
feature_2,-1.616674,0.669025,5.117934,1.508050,-2.998303,0.752382,-0.332134,0.717059,0.298679,-1.236423
feature_3,0.514712,-1.276948,-3.700463,-1.036884,1.793797,0.061186,0.162031,0.102429,-2.520496,3.029865
feature_4,-0.294449,-0.629200,-3.071781,-0.348713,0.138640,0.112934,-4.167281,-3.733083,0.181639,2.231804
feature_5,-0.168076,-0.723463,-0.870718,1.042102,0.215236,1.553327,-1.498722,-2.430186,0.422883,0.434388
feature_6,-0.737200,1.141347,0.541557,1.173786,-0.266201,0.655819,0.044746,-0.410455,2.391613,-4.436130
feature_7,2.205685,-2.237312,0.961470,-0.504714,-1.607260,-1.007727,-2.162316,-2.942299,0.249525,1.747591
feature_8,-0.822867,2.075696,1.130719,-0.408578,-0.450379,-1.329416,0.797892,1.209964,-0.867040,-2.973186
feature_9,-2.207886,0.502049,-2.539167,0.283530,0.112774,1.045324,-3.580595,-2.117530,-0.597481,1.975224


# Tratando o DataFrame

In [6]:
grandom_state = 31415922

# Variavel Dependente
var_dep = 'target'
y = df[var_dep]
x = df.drop(var_dep, axis=1)

############################################################################################
# DUMMYRIZAÇÃO
colunas_categoricas = []
colunas_binarias = []
colunas_mais3_categorias = []
colunas_numericas = []

for coluna in x.columns:
    if df[coluna].dtype == 'O':
        colunas_categoricas.append(coluna)

        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_mais3_categorias.append(coluna)
    else:
        colunas_numericas.append(coluna)

############################################################################################
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_categoricas), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

# Remover prefixo 'remainder__' das colunas que não foram transformadas
#novos_nomes_colunas = [nome.replace('remainder__', '') for nome in novos_nomes_colunas]

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 

############################################################################################
# Normalização (scaling entre 0 e 1) com MinMaxScaler ******************************
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
#x = normalizacao.fit_transform(x)
# df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

# Padronização (média 0 e desvio padrão 1) com StandardScaler **********************
from sklearn.preprocessing import StandardScaler
padronizacao = StandardScaler()
#x = padronizacao.fit_transform(x)
# df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()

############################################################################################
# DEFININDO A VARIAVEL DEPENDENTE
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

############################################################################################
#backups
x_inteiro = x
y_inteiro = y

# DIVIDINDO BASE EM TREINO E TESTE
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = grandom_state) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

print(x_treino.shape, x_teste.shape)

print(y_treino.shape, y_teste.shape)
print(pd.concat([pd.Series(y_treino).value_counts(), pd.Series(y_treino).value_counts(normalize=True)*100], axis=1))
print(pd.concat([pd.Series(y_teste).value_counts(), pd.Series(y_teste).value_counts(normalize=True)*100], axis=1))

(5509, 10) (1837, 10)
(5509,) (1837,)
   count  proportion
0   5210   94.572518
1    299    5.427482
   count  proportion
0   1737   94.556342
1    100    5.443658


In [7]:
# lista_corte = np.concatenate((np.arange(1, 10, 1), np.arange(10, 31, 2)))
lista_corte = np.arange(1, 51, 1)
lista_corte

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

# Implementando Teste

In [8]:
from xgboost import XGBClassifier

modelo = XGBClassifier(random_state=grandom_state)
modelo.fit(x_treino, y_treino)

# y_pred = modelo.predict(x_teste)
# avaliar_modelo(y_teste, y_pred)

y_probs_positivas = modelo.predict_proba(x_teste)[:,1]*100
aval_modelo_corte_tabela(y_teste, y_probs_positivas, pontos_de_corte = lista_corte)['fb_score'].max()

0.364

# Bagging

## Criando modelo treinado com amostras balanceadas

In [9]:
## Melhores modelos de balanceamento de classes:

# Tomek Links
from imblearn.under_sampling import TomekLinks
tomek_links = TomekLinks()
x_treino_balanceado_1, y_treino_balanceado_1 = tomek_links.fit_resample(x_treino, y_treino)
modelo_balanceado_1 = XGBClassifier(random_state=grandom_state); modelo_balanceado_1.fit(x_treino_balanceado_1, y_treino_balanceado_1)
y_probs_positivas_1 = modelo_balanceado_1.predict_proba(x_teste)[:,1]*100; print(aval_modelo_corte_tabela(y_teste, y_probs_positivas_1, pontos_de_corte = lista_corte)['fb_score'].max())

# Edited Nearest Neighbors (ENN)
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()
x_treino_balanceado_2, y_treino_balanceado_2 = enn.fit_resample(x_treino, y_treino)
modelo_balanceado_2 = XGBClassifier(random_state=grandom_state); modelo_balanceado_2.fit(x_treino_balanceado_2, y_treino_balanceado_2)
y_probs_positivas_2 = modelo_balanceado_2.predict_proba(x_teste)[:,1]*100; print(aval_modelo_corte_tabela(y_teste, y_probs_positivas_2, pontos_de_corte = lista_corte)['fb_score'].max())

# Repeated Edited Nearest Neighbours (RENN)
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
renn = RepeatedEditedNearestNeighbours()
x_treino_balanceado_3, y_treino_balanceado_3 = renn.fit_resample(x_treino, y_treino)
modelo_balanceado_3 = XGBClassifier(random_state=grandom_state); modelo_balanceado_3.fit(x_treino_balanceado_3, y_treino_balanceado_3)
y_probs_positivas_3 = modelo_balanceado_3.predict_proba(x_teste)[:,1]*100; print(aval_modelo_corte_tabela(y_teste, y_probs_positivas_3, pontos_de_corte = lista_corte)['fb_score'].max())

# One-Sided Selection (OSS)
from imblearn.under_sampling import OneSidedSelection
oss = OneSidedSelection()
x_treino_balanceado_4, y_treino_balanceado_4 = oss.fit_resample(x_treino, y_treino)
modelo_balanceado_4 = XGBClassifier(random_state=grandom_state); modelo_balanceado_4.fit(x_treino_balanceado_4, y_treino_balanceado_4)
y_probs_positivas_4 = modelo_balanceado_4.predict_proba(x_teste)[:,1]*100; print(aval_modelo_corte_tabela(y_teste, y_probs_positivas_4, pontos_de_corte = lista_corte)['fb_score'].max())

0.406
0.412
0.405
0.381


## Combinando os dois modelo com Bagging

In [10]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Meta-modelo
meta_model = XGBClassifier(random_state=grandom_state)

# Configuração do Stacking
stacking_model = StackingClassifier(
    estimators=[('comum', modelo), 
                ('modelo_balanceado_1', modelo_balanceado_1),
                ('modelo_balanceado_2', modelo_balanceado_2),
                ('modelo_balanceado_3', modelo_balanceado_3),
                ('modelo_balanceado_4', modelo_balanceado_4)],
    final_estimator=meta_model
)

# Treinamento do Stacking
stacking_model.fit(x_treino, y_treino)

y_probs_positivas = stacking_model.predict_proba(x_teste)[:,1]*100
aval_modelo_corte_tabela(y_teste, y_probs_positivas)

,threshold,tn,fp,fn,tp,acuracia,precisao,sensibilidade,especificidade,f1,fb_score,valor_pre_posi,valor_pre_neg,taxa_falsos_positivos,taxa_falsos_negativos,fdr,fo_r,indice_youden,coef_matthews
0,10,1589,148,60,40,0.887,0.213,0.40,0.915,0.278,0.315,0.213,0.964,0.085,0.60,0.787,0.036,0.315,0.236
1,20,1718,19,71,29,0.951,0.604,0.29,0.989,0.392,0.345,0.604,0.960,0.011,0.71,0.396,0.040,0.279,0.397
2,30,1727,10,75,25,0.954,0.714,0.25,0.994,0.370,0.312,0.714,0.958,0.006,0.75,0.286,0.042,0.244,0.405
3,40,1733,4,84,16,0.952,0.800,0.16,0.998,0.267,0.212,0.800,0.954,0.002,0.84,0.200,0.046,0.158,0.345
4,50,1733,4,84,16,0.952,0.800,0.16,0.998,0.267,0.212,0.800,0.954,0.002,0.84,0.200,0.046,0.158,0.345
5,60,1733,4,84,16,0.952,0.800,0.16,0.998,0.267,0.212,0.800,0.954,0.002,0.84,0.200,0.046,0.158,0.345
6,70,1733,4,84,16,0.952,0.800,0.16,0.998,0.267,0.212,0.800,0.954,0.002,0.84,0.200,0.046,0.158,0.345
7,80,1736,1,89,11,0.951,0.917,0.11,0.999,0.196,0.151,0.917,0.951,0.001,0.89,0.083,0.049,0.109,0.308
8,90,1736,1,89,11,0.951,0.917,0.11,0.999,0.196,0.151,0.917,0.951,0.001,0.89,0.083,0.049,0.109,0.308


In [11]:
previsoes = pd.DataFrame({
    'y': y_teste,
    'comum': y_probs_positivas,
    'modelo_balanceado_1': y_probs_positivas_1,
    'modelo_balanceado_2': y_probs_positivas_2,
    'modelo_balanceado_3': y_probs_positivas_3,
    'modelo_balanceado_4': y_probs_positivas_4
    })
previsoes

,y,comum,modelo_balanceado_1,modelo_balanceado_2,modelo_balanceado_3,modelo_balanceado_4
0,0,0.175867,0.017442,0.008839,0.009187,0.009113
1,0,3.673123,0.397072,0.483888,0.287877,1.433235
2,1,2.972302,0.164974,0.066711,0.082416,0.083890
3,0,12.364546,0.253411,3.160205,0.260829,0.788989
4,0,2.203320,1.957071,1.755916,0.335330,0.094957
...,...,...,...,...,...,...
1832,0,7.932042,0.066511,0.028882,0.059421,0.060475
1833,0,4.777137,0.297768,0.734990,0.851600,0.755638
1834,0,0.538712,0.005052,0.008561,0.011344,0.009003
1835,0,12.224514,0.751178,1.068820,3.171382,0.187501


In [12]:
import plotly.express as px
colunas = list(previsoes.columns)
colunas.remove('y')
px.box(previsoes, x=colunas, color="y")

In [13]:
import plotly.graph_objects as go

import plotly.graph_objects as go

fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = previsoes['y'],
                   colorscale = [[0,'grey'], [1,'red']]),
        dimensions = list([
            dict(range = [0, 100],
                label = 'comum', values = previsoes['comum']),
            dict(range = [0, 100],
                label = 'modelo_balanceado_1', values = previsoes['modelo_balanceado_1']),
            dict(range = [0, 100],
                label = 'modelo_balanceado_2', values = previsoes['modelo_balanceado_2']),
            dict(range = [0, 100],
                label = 'modelo_balanceado_3', values = previsoes['modelo_balanceado_3']),
            dict(range = [0, 100],
                label = 'modelo_balanceado_4', values = previsoes['modelo_balanceado_4']),
            dict(range = [0, 1],
                #constraintrange = [0.5,1],
                label = 'y', values = previsoes['y'])
        ])
    )
)
fig.show()
####################################################################################


# Otimizando modelo com maior Fbscore

In [14]:
import optuna
from sklearn.metrics import fbeta_score

lista_corte = np.arange(1, 51, 1)

def objective(trial):
    # Sugerindo um número de estimadores entre 10 e 100
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, 
                                             log=True
                                             #step=0.0001
                                             ),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
    }
 
    modelo = XGBClassifier(**params, random_state=grandom_state, n_jobs=-1)
    modelo.fit(x_treino, y_treino)
 
    # Fazendo previsões
    #predictions = modelo.predict(x_teste)
    #metrica = fbeta_score(y_pred=predictions, y_true=y_teste, beta=1.5)

    y_probs_positivas = modelo.predict_proba(x_teste)[:,1]*100
    tabela = aval_modelo_corte_tabela(y_teste, y_probs_positivas, pontos_de_corte = lista_corte)
    metrica = tabela['fb_score'].max()
 
    return metrica
 
# Criação do estudo para maximizar o recall
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

[I 2024-10-27 14:04:23,141] A new study created in memory with name: no-name-4dabb02e-75af-4c0f-9976-c8a8f54589c3
[I 2024-10-27 14:04:24,025] Trial 0 finished with value: 0.396 and parameters: {'learning_rate': 0.00991313910554645, 'n_estimators': 436}. Best is trial 0 with value: 0.396.
[I 2024-10-27 14:04:25,921] Trial 1 finished with value: 0.387 and parameters: {'learning_rate': 0.0020907289557497927, 'n_estimators': 842}. Best is trial 0 with value: 0.396.
[I 2024-10-27 14:04:27,826] Trial 2 finished with value: 0.382 and parameters: {'learning_rate': 0.19447178581792546, 'n_estimators': 1677}. Best is trial 0 with value: 0.396.
[I 2024-10-27 14:04:29,873] Trial 3 finished with value: 0.385 and parameters: {'learning_rate': 0.0015846851125535374, 'n_estimators': 1003}. Best is trial 0 with value: 0.396.
[I 2024-10-27 14:04:33,368] Trial 4 finished with value: 0.402 and parameters: {'learning_rate': 0.0019210488201319871, 'n_estimators': 1988}. Best is trial 4 with value: 0.402.
[I

In [15]:
import optuna
from sklearn.metrics import fbeta_score

lista_corte = np.arange(1, 51, 1)

def objective(trial):
    # Sugerindo um número de estimadores entre 10 e 100
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, 
                                             log=True
                                             #step=0.0001
                                             ),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
    }
 
    modelo = XGBClassifier(**params, random_state=grandom_state, n_jobs=-1)
    modelo.fit(x_treino_balanceado_1, y_treino_balanceado_1)
 
    # Fazendo previsões
    #predictions = modelo.predict(x_teste)
    #metrica = fbeta_score(y_pred=predictions, y_true=y_teste, beta=1.5)

    y_probs_positivas = modelo.predict_proba(x_teste)[:,1]*100
    tabela = aval_modelo_corte_tabela(y_teste, y_probs_positivas, pontos_de_corte = lista_corte)
    metrica = tabela['fb_score'].max()
 
    return metrica
 
# Criação do estudo para maximizar o recall
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

[I 2024-10-27 14:09:09,558] A new study created in memory with name: no-name-81a3b804-3739-47b5-a3aa-0fa47b7d3a00
[I 2024-10-27 14:09:11,487] Trial 0 finished with value: 0.382 and parameters: {'learning_rate': 0.004246516100048442, 'n_estimators': 971}. Best is trial 0 with value: 0.382.
[I 2024-10-27 14:09:12,880] Trial 1 finished with value: 0.384 and parameters: {'learning_rate': 0.2034764133966098, 'n_estimators': 801}. Best is trial 1 with value: 0.384.
[I 2024-10-27 14:09:14,865] Trial 2 finished with value: 0.378 and parameters: {'learning_rate': 0.04931419199434804, 'n_estimators': 920}. Best is trial 1 with value: 0.384.
[I 2024-10-27 14:09:17,969] Trial 3 finished with value: 0.395 and parameters: {'learning_rate': 0.04322837290046208, 'n_estimators': 1775}. Best is trial 3 with value: 0.395.
[I 2024-10-27 14:09:18,904] Trial 4 finished with value: 0.379 and parameters: {'learning_rate': 0.09468922359156641, 'n_estimators': 534}. Best is trial 3 with value: 0.395.
[I 2024-10

In [16]:
import optuna
from sklearn.metrics import fbeta_score

lista_corte = np.arange(1, 51, 1)

def objective(trial):
    # Sugerindo um número de estimadores entre 10 e 100
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, 
                                             log=True
                                             #step=0.0001
                                             ),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
    }
 
    modelo = XGBClassifier(**params, random_state=grandom_state, n_jobs=-1)
    modelo.fit(x_treino_balanceado_2, y_treino_balanceado_2)
 
    # Fazendo previsões
    #predictions = modelo.predict(x_teste)
    #metrica = fbeta_score(y_pred=predictions, y_true=y_teste, beta=1.5)

    y_probs_positivas = modelo.predict_proba(x_teste)[:,1]*100
    tabela = aval_modelo_corte_tabela(y_teste, y_probs_positivas, pontos_de_corte = lista_corte)
    metrica = tabela['fb_score'].max()
 
    return metrica
 
# Criação do estudo para maximizar o recall
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

[I 2024-10-27 14:13:48,422] A new study created in memory with name: no-name-f37a118c-42c7-4352-a258-1df2cd8da15a
[I 2024-10-27 14:13:50,628] Trial 0 finished with value: 0.422 and parameters: {'learning_rate': 0.04956595850667763, 'n_estimators': 1224}. Best is trial 0 with value: 0.422.
[I 2024-10-27 14:13:52,548] Trial 1 finished with value: 0.388 and parameters: {'learning_rate': 0.0015433135208826896, 'n_estimators': 325}. Best is trial 0 with value: 0.422.
[I 2024-10-27 14:13:55,961] Trial 2 finished with value: 0.418 and parameters: {'learning_rate': 0.04340805068310417, 'n_estimators': 1502}. Best is trial 0 with value: 0.422.
[I 2024-10-27 14:13:56,616] Trial 3 finished with value: 0.385 and parameters: {'learning_rate': 0.004043769778098151, 'n_estimators': 245}. Best is trial 0 with value: 0.422.
[I 2024-10-27 14:13:58,752] Trial 4 finished with value: 0.406 and parameters: {'learning_rate': 0.012308919586083317, 'n_estimators': 1144}. Best is trial 0 with value: 0.422.
[I 2

In [17]:
import optuna
from sklearn.metrics import fbeta_score

lista_corte = np.arange(1, 51, 1)

def objective(trial):
    # Sugerindo um número de estimadores entre 10 e 100
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, 
                                             log=True
                                             #step=0.0001
                                             ),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
    }
 
    modelo = XGBClassifier(**params, random_state=grandom_state, n_jobs=-1)
    modelo.fit(x_treino_balanceado_3, y_treino_balanceado_3)
 
    # Fazendo previsões
    #predictions = modelo.predict(x_teste)
    #metrica = fbeta_score(y_pred=predictions, y_true=y_teste, beta=1.5)

    y_probs_positivas = modelo.predict_proba(x_teste)[:,1]*100
    tabela = aval_modelo_corte_tabela(y_teste, y_probs_positivas, pontos_de_corte = lista_corte)
    metrica = tabela['fb_score'].max()
 
    return metrica
 
# Criação do estudo para maximizar o recall
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

[I 2024-10-27 14:19:26,555] A new study created in memory with name: no-name-3729c488-3503-4731-8044-dd4b3af63046
[I 2024-10-27 14:19:29,341] Trial 0 finished with value: 0.404 and parameters: {'learning_rate': 0.07033522482709617, 'n_estimators': 1152}. Best is trial 0 with value: 0.404.
[I 2024-10-27 14:19:30,181] Trial 1 finished with value: 0.377 and parameters: {'learning_rate': 0.0013859831809185676, 'n_estimators': 237}. Best is trial 0 with value: 0.404.
[I 2024-10-27 14:19:35,267] Trial 2 finished with value: 0.389 and parameters: {'learning_rate': 0.008585432640057383, 'n_estimators': 1986}. Best is trial 0 with value: 0.404.
[I 2024-10-27 14:19:39,364] Trial 3 finished with value: 0.394 and parameters: {'learning_rate': 0.0029253942731351035, 'n_estimators': 1452}. Best is trial 0 with value: 0.404.
[I 2024-10-27 14:19:42,109] Trial 4 finished with value: 0.388 and parameters: {'learning_rate': 0.0016420009245286893, 'n_estimators': 917}. Best is trial 0 with value: 0.404.
[

In [18]:
import optuna
from sklearn.metrics import fbeta_score

lista_corte = np.arange(1, 51, 1)

def objective(trial):
    # Sugerindo um número de estimadores entre 10 e 100
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, 
                                             log=True
                                             #step=0.0001
                                             ),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
    }
 
    modelo = XGBClassifier(**params, random_state=grandom_state, n_jobs=-1)
    modelo.fit(x_treino_balanceado_4, y_treino_balanceado_4)
 
    # Fazendo previsões
    #predictions = modelo.predict(x_teste)
    #metrica = fbeta_score(y_pred=predictions, y_true=y_teste, beta=1.5)

    y_probs_positivas = modelo.predict_proba(x_teste)[:,1]*100
    tabela = aval_modelo_corte_tabela(y_teste, y_probs_positivas, pontos_de_corte = lista_corte)
    metrica = tabela['fb_score'].max()
 
    return metrica
 
# Criação do estudo para maximizar o recall
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

[I 2024-10-27 14:22:41,025] A new study created in memory with name: no-name-de7233a4-7bcd-48ef-9eb4-8149ac4bc28d
[I 2024-10-27 14:22:42,111] Trial 0 finished with value: 0.385 and parameters: {'learning_rate': 0.004175866703515201, 'n_estimators': 648}. Best is trial 0 with value: 0.385.
[I 2024-10-27 14:22:42,512] Trial 1 finished with value: 0.331 and parameters: {'learning_rate': 0.0012944101329927106, 'n_estimators': 189}. Best is trial 0 with value: 0.385.
[I 2024-10-27 14:22:43,457] Trial 2 finished with value: 0.376 and parameters: {'learning_rate': 0.2669456625615322, 'n_estimators': 975}. Best is trial 0 with value: 0.385.
[I 2024-10-27 14:22:45,059] Trial 3 finished with value: 0.399 and parameters: {'learning_rate': 0.05458441498660264, 'n_estimators': 1319}. Best is trial 3 with value: 0.399.
[I 2024-10-27 14:22:45,522] Trial 4 finished with value: 0.404 and parameters: {'learning_rate': 0.06017010415180804, 'n_estimators': 236}. Best is trial 4 with value: 0.404.
[I 2024-